In [1]:
import pandas as pd
import numpy as np
import re
import pickle

import sys
sys.path.append('../')

import src.support as sp

In [27]:
originals = pd.read_csv("../data/netflix_originals.csv", index_col = 0)

In [28]:
originals.head()

,Title,Genre,Premiere,Runtime,IMDB Score,Language
0,Enter the Anime,Documentary,"August 5, 2019",58,2.5,English/Japanese
1,Dark Forces,Thriller,"August 21, 2020",81,2.6,Spanish
2,The App,Science fiction/Drama,"December 26, 2019",79,2.6,Italian
3,The Open House,Horror thriller,"January 19, 2018",94,3.2,English
4,Kaali Khuhi,Mystery,"October 30, 2020",90,3.4,Hindi


In [29]:
originals.shape

(584, 6)

In [30]:
df= pd.read_csv("../data/netflix_titles.csv")
df.head(1)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."


In [31]:
originals_df = df.merge(originals, left_on= "title", right_on="Title", how="inner")

In [32]:
originals_df.drop(['release_year', 'rating', 'duration', 'listed_in', 'description','Title', "date_added"], axis = 1, inplace=True)

In [33]:
originals_df.head(2)

,show_id,type,title,director,cast,country,Genre,Premiere,Runtime,IMDB Score,Language
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,Documentary,"October 2, 2020",90,7.5,English
1,s142,Movie,Extraction,Steven C. Miller,"Bruce Willis, Kellan Lutz, Gina Carano, D.B. S...","United States, United Kingdom, Canada",Action,"April 24, 2020",117,6.7,English


In [34]:
originals_df2 = originals_df.dropna()

In [35]:

originals_df2.isnull().sum()

show_id       0
type          0
title         0
director      0
cast          0
country       0
Genre         0
Premiere      0
Runtime       0
IMDB Score    0
Language      0
dtype: int64

In [36]:
originals_df["cast"].str.contains("Adam Sandler", regex = True)

0        NaN
1      False
2      False
3      False
4      False
       ...  
508    False
509      NaN
510      NaN
511    False
512    False
Name: cast, Length: 513, dtype: object

In [37]:
originals_df2[originals_df2["cast"].str.contains("Adam Sandler", regex = True)]

,show_id,type,title,director,cast,country,Genre,Premiere,Runtime,IMDB Score,Language
117,s1880,Movie,Hubie Halloween,Steve Brill,"Adam Sandler, Kevin James, Julie Bowen, Ray Li...",United States,Comedy,"October 7, 2020",103,5.2,English
290,s3754,Movie,Murder Mystery,Kyle Newacheck,"Adam Sandler, Jennifer Aniston, Luke Evans, Ge...",United States,Comedy mystery,"June 14, 2019",97,6.0,English
393,s4913,Movie,The Week Of,Robert Smigel,"Adam Sandler, Chris Rock, Steve Buscemi, Rache...",United States,Comedy,"April 27, 2018",116,5.2,English
431,s5228,Movie,The Meyerowitz Stories (New and Selected),Noah Baumbach,"Adam Sandler, Ben Stiller, Dustin Hoffman, Eli...",United States,Comedy-drama,"October 13, 2017",112,6.9,English
463,s5534,Movie,Sandy Wexler,Steven Brill,"Adam Sandler, Jennifer Hudson, Kevin James, Te...",United States,Comedy,"April 14, 2017",131,5.2,English
498,s5847,Movie,The Do-Over,Steven Brill,"Adam Sandler, David Spade, Paula Patton, Kathr...",United States,Action comedy,"May 27, 2016",108,5.7,English
503,s5880,Movie,The Ridiculous 6,Frank Coraci,"Adam Sandler, Terry Crews, Jorge Garcia, Taylo...",United States,Western,"December 11, 2015",119,4.8,English


# Cleaning

## Country

In [70]:
originals_df.isnull().sum()

show_id        0
type           0
title          0
director      22
cast          91
country       14
Genre          0
Premiere       0
Runtime        0
IMDB Score     0
Language       0
dtype: int64

In [71]:
originals_df["country"] = originals_df["country"].apply(lambda x: x.split(",")[0] if type(x) == str else x)
originals_df.head(1)

,show_id,type,title,director,cast,country,Genre,Premiere,Runtime,IMDB Score,Language
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,Documentary,"October 2, 2020",90,7.5,English


## Language

In [72]:
originals_df["Language"] = originals_df["Language"].apply(lambda x: x.split("/")[0] if type(x) == str else x)
originals_df.head(2)

,show_id,type,title,director,cast,country,Genre,Premiere,Runtime,IMDB Score,Language
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,Documentary,"October 2, 2020",90,7.5,English
1,s142,Movie,Extraction,Steven C. Miller,"Bruce Willis, Kellan Lutz, Gina Carano, D.B. S...",United States,Action,"April 24, 2020",117,6.7,English


## Cast

In [11]:
originals_df.head()

,show_id,type,title,director,cast,country,Genre,Premiere,Runtime,IMDB Score,Language
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,Documentary,"October 2, 2020",90,7.5,English
1,s142,Movie,Extraction,Steven C. Miller,"Bruce Willis, Kellan Lutz, Gina Carano, D.B. S...","United States, United Kingdom, Canada",Action,"April 24, 2020",117,6.7,English
2,s625,Movie,Prime Time,Jakub Piątek,"Bartosz Bielenia, Magdalena Popławska, Andrzej...",NaN,Thriller,"April 14, 2021",91,5.7,Polish
3,s835,Movie,Blue Miracle,Julio Quintana,"Jimmy Gonzales, Dennis Quaid, Anthony Gonzalez...",United States,Drama,"May 27, 2021",95,6.7,English
4,s837,Movie,Ghost Lab,Paween Purijitpanya,"Thanapob Leeratanakachorn, Paris Intarakomalya...",Thailand,Horror,"May 26, 2021",117,5.2,Thai


In [51]:
originals_df["cast"] = originals_df["cast"].apply(lambda x: x.split(",") if type(x) == str else x)
all_cast = originals_df.explode("cast")
all_cast["cast"] = all_cast["cast"].str.strip()

In [52]:
originals_df.head()

,show_id,type,title,director,cast,country,Genre,Premiere,Runtime,IMDB Score,Language
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,Documentary,"October 2, 2020",90,7.5,English
1,s142,Movie,Extraction,Steven C. Miller,"[Bruce Willis, Kellan Lutz, Gina Carano, D....","United States, United Kingdom, Canada",Action,"April 24, 2020",117,6.7,English
2,s625,Movie,Prime Time,Jakub Piątek,"[Bartosz Bielenia, Magdalena Popławska, Andr...",NaN,Thriller,"April 14, 2021",91,5.7,Polish
3,s835,Movie,Blue Miracle,Julio Quintana,"[Jimmy Gonzales, Dennis Quaid, Anthony Gonza...",United States,Drama,"May 27, 2021",95,6.7,English
4,s837,Movie,Ghost Lab,Paween Purijitpanya,"[Thanapob Leeratanakachorn, Paris Intarakomal...",Thailand,Horror,"May 26, 2021",117,5.2,Thai


In [53]:
all_cast.head(3)

,show_id,type,title,director,cast,country,Genre,Premiere,Runtime,IMDB Score,Language
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,Documentary,"October 2, 2020",90,7.5,English
1,s142,Movie,Extraction,Steven C. Miller,Bruce Willis,"United States, United Kingdom, Canada",Action,"April 24, 2020",117,6.7,English
1,s142,Movie,Extraction,Steven C. Miller,Kellan Lutz,"United States, United Kingdom, Canada",Action,"April 24, 2020",117,6.7,English


In [55]:
all_cast[all_cast["cast"] == "Robbie Amell"]

,show_id,type,title,director,cast,country,Genre,Premiere,Runtime,IMDB Score,Language
133,s2030,Movie,The Babysitter: Killer Queen,McG,Robbie Amell,United States,Comedy/Horror,"September 10, 2020",102,5.8,English
168,s2295,Movie,Desperados,LP,Robbie Amell,United States,Romantic comedy,"July 3, 2020",106,5.2,English
412,s5045,Movie,When We First Met,Ari Sandel,Robbie Amell,United States,Romantic comedy,"February 9, 2018",97,6.4,English
430,s5227,Movie,The Babysitter,McG,Robbie Amell,United States,Teen comedy horror,"October 13, 2017",85,6.3,English
488,s5793,Movie,ARQ,Tony Elliott,Robbie Amell,"Canada, United States",Science fiction/Thriller,"September 16, 2016",88,6.4,English


In [56]:
all_cast[all_cast["title"] == "ARQ"]

,show_id,type,title,director,cast,country,Genre,Premiere,Runtime,IMDB Score,Language
488,s5793,Movie,ARQ,Tony Elliott,Robbie Amell,"Canada, United States",Science fiction/Thriller,"September 16, 2016",88,6.4,English
488,s5793,Movie,ARQ,Tony Elliott,Rachael Taylor,"Canada, United States",Science fiction/Thriller,"September 16, 2016",88,6.4,English
488,s5793,Movie,ARQ,Tony Elliott,Shaun Benson,"Canada, United States",Science fiction/Thriller,"September 16, 2016",88,6.4,English
488,s5793,Movie,ARQ,Tony Elliott,Gray Powell,"Canada, United States",Science fiction/Thriller,"September 16, 2016",88,6.4,English
488,s5793,Movie,ARQ,Tony Elliott,Jacob Neayem,"Canada, United States",Science fiction/Thriller,"September 16, 2016",88,6.4,English
488,s5793,Movie,ARQ,Tony Elliott,Adam Butcher,"Canada, United States",Science fiction/Thriller,"September 16, 2016",88,6.4,English
488,s5793,Movie,ARQ,Tony Elliott,Tantoo Cardinal,"Canada, United States",Science fiction/Thriller,"September 16, 2016",88,6.4,English


In [57]:
all_cast[all_cast["cast"] == "Maya Rudolph"]

,show_id,type,title,director,cast,country,Genre,Premiere,Runtime,IMDB Score,Language
106,s1794,Movie,Sarah Cooper: Everything's Fine,Natasha Lyonne,Maya Rudolph,United States,Variety show,"October 27, 2020",49,5.6,English
117,s1880,Movie,Hubie Halloween,Steve Brill,Maya Rudolph,United States,Comedy,"October 7, 2020",103,5.2,English
197,s2657,Movie,The Willoughbys,Kris Pearn,Maya Rudolph,"Canada, United States, United Kingdom",Animation/Comedy/Adventure,"April 22, 2020",90,6.4,English
297,s3803,Movie,The Lonely Island Presents: The Unauthorized B...,"Mike Diva, Akiva Schaffer",Maya Rudolph,NaN,Comedy / Musical,"May 23, 2019",30,6.9,English
302,s3842,Movie,Wine Country,Amy Poehler,Maya Rudolph,United States,Comedy,"May 10, 2019",103,5.5,English
504,s5882,Movie,A Very Murray Christmas,Sofia Coppola,Maya Rudolph,United States,Comedy / Musical,"December 4, 2015",56,5.5,English


In [58]:
all_cast[all_cast["cast"] == "Adam Sandler"]

,show_id,type,title,director,cast,country,Genre,Premiere,Runtime,IMDB Score,Language
117,s1880,Movie,Hubie Halloween,Steve Brill,Adam Sandler,United States,Comedy,"October 7, 2020",103,5.2,English
290,s3754,Movie,Murder Mystery,Kyle Newacheck,Adam Sandler,United States,Comedy mystery,"June 14, 2019",97,6.0,English
393,s4913,Movie,The Week Of,Robert Smigel,Adam Sandler,United States,Comedy,"April 27, 2018",116,5.2,English
431,s5228,Movie,The Meyerowitz Stories (New and Selected),Noah Baumbach,Adam Sandler,United States,Comedy-drama,"October 13, 2017",112,6.9,English
463,s5534,Movie,Sandy Wexler,Steven Brill,Adam Sandler,United States,Comedy,"April 14, 2017",131,5.2,English
498,s5847,Movie,The Do-Over,Steven Brill,Adam Sandler,United States,Action comedy,"May 27, 2016",108,5.7,English
503,s5880,Movie,The Ridiculous 6,Frank Coraci,Adam Sandler,United States,Western,"December 11, 2015",119,4.8,English


In [63]:
all_cast.groupby("cast").size().reset_index().sort_values(by = 0, ascending = False)

,cast,0
29,Adam Sandler,7
1854,Maya Rudolph,6
171,Andrew Bachelor,5
2333,Robbie Amell,5
1201,Jason Schwartzman,4
...,...,...
1024,Hakim Jemili,1
1025,Hal Landon Jr.,1
1026,Hala Finley,1
1028,Haley Reinhart,1


In [60]:
top10_actors = all_cast.groupby("cast").size().reset_index().sort_values(by = 0, ascending = False)["cast"].iloc[:10].to_list()
top10_actors

['Adam Sandler',
 'Maya Rudolph',
 'Andrew Bachelor',
 'Robbie Amell',
 'Jason Schwartzman',
 'Zachary Quinto',
 'David Spade',
 'Rob Schneider',
 'Anthony Mackie',
 'Ken Marino']

In [93]:

with open('../data/pickle/top10_actors.pickle', 'wb') as handle:
    pickle.dump(top10_actors, handle)

## Director

In [61]:
top10_directors = all_cast.groupby("director").size().reset_index().sort_values(by = 0, ascending = False)["director"].iloc[:10].to_list()
top10_directors

['McG',
 'John Schultz',
 'Amy Poehler',
 'Christopher Guest',
 'Steve Brill',
 'Kyle Newacheck',
 'Javier Colinas',
 'Frank Coraci',
 'Damien O’Connor',
 'Robert Rodriguez']

In [62]:
with open ('../data/pickle/top10_directors.pickle', "wb") as dir:
    pickle.dump(top10_directors, dir)

## Genre

In [94]:
originals_df["Genre"] = originals_df["Genre"].apply(lambda x: x.split("/")[0].split("-")[0].replace("film", "").replace("thriller", "").strip() if type(x) == str else x)
originals_df.head(2)

,show_id,type,title,director,cast,country,Genre,Premiere,Runtime,IMDB Score,Language
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,Documentary,"October 2, 2020",90,7.5,English
1,s142,Movie,Extraction,Steven C. Miller,"[Bruce Willis, Kellan Lutz, Gina Carano, D....",United States,Action,"April 24, 2020",117,6.7,English


In [95]:
def clean_genre(x):
    
    drama = re.findall("drama", x)
    romantic = re.findall("Roman", x)
    dance = re.findall("[D|d]ance|[M|m]us",x)
    comedy = re.findall("[C|c]ome", x)
    nulos = re.findall("^$|One", x)

    if drama:
        return ""
    elif dance:
        return "Musical"
    elif romantic:
        return "Romantic"
    elif comedy:
        return "Comedy"
    
    elif nulos:
        return np.nan
    else:
        return x

    

In [96]:
originals_df["Genre"] = originals_df["Genre"].apply(clean_genre)

In [50]:
originals_df.head()

,show_id,type,title,director,cast,country,Genre,Premiere,Runtime,IMDB Score,Language
1,s142,Movie,Extraction,Steven C. Miller,Bruce Willis,United States,Action,"April 24, 2020",117,6.7,English
1,s142,Movie,Extraction,Steven C. Miller,Kellan Lutz,United States,Action,"April 24, 2020",117,6.7,English
1,s142,Movie,Extraction,Steven C. Miller,Gina Carano,United States,Action,"April 24, 2020",117,6.7,English
1,s142,Movie,Extraction,Steven C. Miller,D.B. Sweeney,United States,Action,"April 24, 2020",117,6.7,English
1,s142,Movie,Extraction,Steven C. Miller,Joshua Mikel,United States,Action,"April 24, 2020",117,6.7,English


In [60]:
originals_df.groupby(["director"])["IMDB Score"].size().reset_index().sort_values(by = "IMDB Score", ascending = False)

,director,IMDB Score
238,McG,43
181,John Schultz,29
341,Steve Brill,23
17,Amy Poehler,23
205,Kyle Newacheck,22
...,...,...
186,Juan Carlos Rulfo,1
95,David Sampliner,1
355,Thom Zimny,1
357,Tom Donahue,1


In [20]:
originals_df.to_csv("../data/netflix_originals_clean.csv")